<a href="https://colab.research.google.com/github/alexlautw9527/for_notebook/blob/main/%E9%89%85%E4%BA%A8%E7%B6%B2%E6%96%87%E5%AD%97%E6%8E%A2%E5%8B%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
%%capture
!pip install transformers
!pip install gdown
!pip install opencc-python-reimplemented

In [24]:
import torch
import numpy as np
import pandas as pd

In [76]:
from opencc import OpenCC

cc = OpenCC('t2s')

In [2]:
!gdown https://drive.google.com/uc?id=1qW1YWtw3q9Q28QThrIY-rDU9Gl-SLIKO
!unzip /content/FinBERT_L-12_H-768_A-12_pytorch.zip

Downloading...
From: https://drive.google.com/uc?id=1qW1YWtw3q9Q28QThrIY-rDU9Gl-SLIKO
To: /content/FinBERT_L-12_H-768_A-12_pytorch.zip
100% 382M/382M [00:02<00:00, 173MB/s]


# 財金BERT

In [12]:
from transformers import BertModel, BertTokenizer

PATH = '/content/FinBERT_L-12_H-768_A-12_pytorch'

In [14]:
tokenizer = BertTokenizer.from_pretrained(PATH ,local_files_only=True)
model = BertModel.from_pretrained(PATH ,local_files_only=True)

Some weights of the model checkpoint at /content/FinBERT_L-12_H-768_A-12_pytorch were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [64]:
sector_dict = {
'健護': '醫療保健服務類股',
'科技': '科技類股', 
'公用事業': '公用事業類股',
'生技':'生技類股', 
'能源':'能源類股',
'替代能源':'替代能源類股',
'消費':'消費品類股',
'金融':'金融類股', 
'礦業':'礦業',
'天然資源':'天然資源', 
'貴金屬現貨':'貴金屬',
'農產品':'農產品',
'貴金屬類股':'貴金屬',
}

In [65]:
mkt_dict= [
'新興市場債',
'高收益債',
'全球債',
'投資等級公司債',
'美國公債',
'亞債',
'REITs不動產投資信託',
'新臺幣類貨幣',
'外幣類貨幣',
'亞太股市', '美國股市', '全球新興市場', 
'全球型股市', '大中華股市', '中國A股', '印度股市','歐洲股市', 
'東南亞股市', '拉丁美洲股市', 
'韓國股市', '新興歐洲股市', '德國股市', '台灣股市','俄羅斯股市',
'中國H股', '日本股市', '英國股市','泰國股市',
'香港股市', '澳洲股市', '印尼股市', '巴西股市', '歐非中東股市'
]


In [66]:
mkt_dict = {x:x for x in mkt_dict}
mkt_dict['亞債'] = '亞洲債券'
mkt_dict['REITs不動產投資信託'] = '不動產'

In [67]:
total_sec_dict = {**sector_dict, **mkt_dict}

In [70]:
sentences = list(set(total_sec_dict.values()))

In [78]:
sentences = [cc.convert(x) for x in sentences]

In [71]:
inputs = tokenizer(sentences, padding=True, return_tensors="pt")


In [79]:
## 單純forward pass 沒要計算梯度
with torch.no_grad():
  outputs = model(**inputs)

In [88]:
x = outputs['last_hidden_state']
np.savetxt('bert_mkt_emb.tsv', x[:,0].cpu().detach().numpy(), delimiter="\t")
pd.DataFrame(sentences).to_csv('bert_mkt_emb_meta.tsv', header=False, index=False, sep="\t")

In [89]:
x[:,0].shape

torch.Size([45, 768])

In [90]:
x[:,0].mean(axis=1).cpu().detach().numpy().shape

(45,)

# 鉅亨網爬蟲


In [92]:
import requests

In [93]:
res = requests.get('https://news.cnyes.com/news/cat/wd_stock')

In [106]:
#res2 = requests.get('https://blog.anuefund.com/page/2/?utm_campaign=bloglink&utm_medium=fund_mainlist&utm_source=anuefundapp')

In [109]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(res.text, 'html.parser')

In [110]:
news_list = soup.find_all("a", attrs={"data-exp-project":"news-title"})

In [ ]:
pd.DataFrame()

In [119]:
[x.find('h3').text for x in news_list]

['美股今年以來已漲25% 策略師：通常這代表12月及明年還會衝一波',
 '中國產尿素將運抵南韓 短缺危機可望解除',
 '〈金融風險指數〉通膨效應升溫 海外信用風險攀升',
 '車用晶片短缺逐步緩解 車廠+零件廠同步拉產能',
 'WHO警告 2022年注射針筒恐短少20億支',
 '美股史上第六大IPO將登場！特斯拉勁敵Rivian完成定價 估募資120億美元',
 '〈財報〉蔚來汽車Q4交車量、營收預測悲觀 盤後挫逾2%',
 'PC用LCD面板行情下滑 2年半以來首見',
 '半導體業砸錢擴產 為何成熟製程晶片荒估延燒到2024年？',
 '拜登積極減碳 將要求航空業2050年以前實現淨零排放',
 '〈美股熱門股〉PayPal第四季財測黯淡跌10% AMC營運逆風重挫11%',
 '蘋果聚焦醫療保健服務 嬌生CEO將加入董事會',
 '美股重點新聞摘要2021年11月10日',
 '美商務部長：供應鏈瓶頸將延續至明年 但正在逐步改善',
 '馬斯克擬賣股消息發酵 特斯拉兩日來市值蒸發近2000億美元',
 '福斯汽車擬在德蓋新廠 力拚特斯拉超級工廠',
 '〈美股盤後〉特斯拉崩落近12% 四大指數盡墨',
 '拜習會即將登場！彭博：線上峰會將於下周舉行',
 '拜登擬斥資數十億美元升級老舊港口 強化美國供應鏈',
 '拜登延長中國軍方企業投資禁令',
 '富人稅是藉口？大賣空本尊爆馬斯克想賣股背後主因',
 '蘋果執行長庫克挺加密貨幣 不僅持有也感興趣',
 '美商務部長雷蒙多：將盡快與英、日展開鋼鋁關稅磋商',
 '美國10月PPI年增8.6% 續創近11年最高',
 '〈美股早盤〉美10月PPI仍處高位 美股漲跌不一、GE漲逾6%',
 '鉅亨美股雷達：Lucid令市場憂心蔚來前景、Meta傳將設實體店介紹元宇宙概念',
 'GE將分拆為3家上市公司 分別專注航空、醫療保健和能源業務',
 '受供應鏈中斷打擊 德國出口連兩月下滑',
 '〈美股盤前要聞〉美公債殖利率落至1.457% 美股期貨漲跌不一',
 '輝達秋季GTC大會 聚焦Omniverse資料生成引擎、虛擬化身技術']

In [118]:
news_list[0]['href']

'/news/id/4767027?exp=a'

In [115]:
news_list[0].find('time').text

'13:40'